# W266 Final Project - Load_DataCleansing_TextProcessing

In [35]:
# Initial data load, summary, and high level sanity checks

# Import a bunch of libraries.
import sys
import glob
import time
from IPython.core.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import brown
nltk.download('brown')
from nltk.corpus import webtext
#import parseyface

reviews = pd.read_csv('RR_Pared_All_Active_2018-07-09.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mfrazzini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /home/mfrazzini/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [46]:
print(reviews.shape)
print(reviews.iloc[323248])

(369888, 8)
MODELID           277297
TESTIMONIALID    1566719
TESTIMONIAL             
OVERALL              NaN
ORGANIZATION           7
APPEARANCE            10
PRICE_VALUE            9
DURABILITY           NaN
Name: 323248, dtype: object


In [37]:
#Remove html special characters that end up in the review text
from bs4 import BeautifulSoup 


Spell Check Procedures with Custom Dictionary Adds

In [38]:
brands = pd.read_csv('Brands_All_Active_2018-07-09.csv')
features = pd.read_csv('FeatureDump_All_Active_2018-07-09T16_31_17.csv')

import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(brown.words())

#Add custom dictionary brands
for index, row in brands.iterrows():
    parsed_words = words(row['BRAND'])
    for w in parsed_words:
        if len(w) >=2:
            WORDS[w] += 1

#Add custom dictionary features
for index, row in features.iterrows():
    feature_blurb = row['FEATURENAME']
    if type(feature_blurb) is str: 
        nopunc = ''
        nopunc = [char for char in feature_blurb if char not in string.punctuation]
        nopunc = ''.join(nopunc)
        parsed_words = words(nopunc)
        for w in parsed_words:
            if len(w) >=2:
                WORDS[w] += 1

#Add custom dictionary collections
WORDS["tls"] += 1

#Add custom dictionary travel words
WORDS["carryon"] += 1
WORDS["toiletry"] += 1
WORDS["laptop"] += 1
WORDS["carousel"] += 1
WORDS["kindle"] += 1
WORDS["ipad"] += 1
WORDS["unpacked"] += 1
WORDS["mac"] += 1
WORDS["iphone"] += 1
WORDS["dvd"] += 1
WORDS["tsa"] += 1

#Add custom dictionary Misc
WORDS["downside"] += 1
WORDS["andor"] += 1
WORDS["classy"] += 1
WORDS["online"] += 1
WORDS["bike"] += 1
WORDS["hassle"] += 1


def dict_check(word):
    if word.lower() in WORDS:
        return(word)
    else:
        return ""

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [39]:
# Test mod for initial dictionary check
print(dict_check("comfortable"))
print(dict_check("confortable"))
print(correction("confortable"))
print(dict_check("Baggalini"))
print(correction("unpacked"))

comfortable

comfortable

unpacked


In [40]:
import csv

def text_process(text):
    #Takes in a string of text, then performs the following:
    #1. Remove all html special characters, with regex to pull partially remove html special characters
    #2. Spelling correction
    #3. Remove all punctuation
    #4. Remove all stopwords
    #5. Stemming of words
    #6. Return the cleaned text as a list of words
    if type(text) is str and len(text) <= 45:
        review = []
        nopunc = ''
        nopunc = [char for char in text if char not in string.punctuation]
        nopunc = ''.join(nopunc)
        for word in nopunc.split():
            #print(word)
            total_words[0] += 1
            if word.lower() not in stopwords.words('english') and len(word) <= int(45):
                #html_decoded = BeautifulSoup(word, "lxml")
                #html_decoded_string = html_decoded.string
                if re.search('\w(\d+)\w', word): #html_decoded_string):
                    word = word.replace("34","\"")
                    word = word.replace("39","\'")
                if word == dict_check(word):
                    correct[0] += 1
                    spell_corrected = word
                else:
                    if word in corrections:
                        spell_corrected = corrections[word]
                        corrections[word] = spell_corrected
                        corrections_count[spell_corrected + ':' + word] += 1
                        corrected[0] += 1
                    else:
                        spell_corrected = correction(word)
                        if word != spell_corrected:
                            corrections[word] = spell_corrected
                            corrections_count[spell_corrected + ':' + word] += 1
                            corrected[0] += 1  
                        else:
                            corrections[word] = word
                            corrected_default[0] += 1
                #porter = PorterStemmer()
                #stemmed = porter.stem(spell_corrected)
                review.append(spell_corrected) 
            else:
                review.append(word)
        return ' '.join(review)
    else:
        return ''

total_reviews = [0]
total_words = [0]
correct = [0]
corrected = [0]
corrected_default = [0]
corrections = dict()
corrections_count = Counter()

processed_testimonial = str

#f = open('debug.txt','w')
    
for index, row in reviews.iterrows():
    total_reviews = index
    clear_output()
    print("On record: {}; ".format(index), "total words", total_words, "words correct: ", correct, \
          "corrected: ", corrected, "corrected default:", corrected_default, end="\r")
    reviews.at[index, 'TESTIMONIAL'] = text_process(row['TESTIMONIAL'])

print("Total reviews: ", total_reviews)
print("Total words: ", total_words)    
    
#word_vectorizer = CountVectorizer(ngram_range=(3,6), analyzer='word', stop_words=None)
#sparse_matrix = word_vectorizer.fit_transform(reviews['TESTIMONIAL'])
#frequencies = sum(sparse_matrix).data

#result_group = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
#result_sort_group = result_group.sort_values('frequency', ascending=False)
#result_sort_group.to_csv("Baseline_ngram_output_w_spellcorrect.csv")


Total reviews:  369887
Total words:  [315335]


KeyboardInterrupt: 

In [41]:
print(processed_testimonial)
corrections_count.most_common()[:-5] 

<class 'str'>


[('slot:alot', 92),
 ('a:Na', 89),
 ("I've:I've", 79),
 ('I:Im', 79),
 ("I'm:I'm", 78),
 ('price:pricey', 64),
 ('ve:Ive', 56),
 ('wellmade:wellmade', 55),
 ('Haven:Havent', 51),
 ('a:na', 46),
 ('haven:havent', 42),
 ('useful:usefull', 40),
 ('a:¿', 38),
 ('Excellent:Excellant', 36),
 ('uv:Luv', 32),
 ('bag:ebag', 30),
 ('Excellent:Excelent', 30),
 ('awesome:awsome', 20),
 ('excellent:excelent', 19),
 ('price:pricy', 18),
 ('welldesigned:welldesigned', 18),
 ('to:¿¿', 17),
 ('excellent:excellant', 16),
 ('comfortable:confortable', 15),
 ('backpack:backback', 14),
 ('didn:didnt', 14),
 ('uv:luv', 13),
 ('in:im', 13),
 ('give:ive', 12),
 ('goodlooking:goodlooking', 12),
 ('convenient:convinient', 12),
 ('advertised:advertized', 11),
 ('17:17"', 11),
 ('Excellent:Exellent', 10),
 ('battery:bagvery', 10),
 ('little:alittle', 10),
 ('Excellent:Excelente', 10),
 ('Did:Didnt', 9),
 ('Wellman:Wellmade', 9),
 ('good:goog', 9),
 ('everyday:everday', 9),
 ('girlfriend:girlfriend', 9),
 ('rest:Gr

In [76]:
import csv

with open('corrections_count_latest.csv','w') as csvfile:
    fieldnames=['spell_correction','count']
    writer=csv.writer(csvfile)
    writer.writerow(fieldnames)
    for key, value in corrections_count.items():
        writer.writerow(list(key) + [value]) 